# Download replays from Ballchaser website

In [8]:
from typing import Dict

In [12]:
import datetime
from ballchaser.client import BallChaser

ball_chaser = BallChaser("AWSZ0JiezKKjPVHQFXNQiPImI6YIKobD6eogELef", backoff=True, max_tries=5)

# search and retrieve replay metadata
replays = [
    replay
    for replay in ball_chaser.list_replays(player_name="GarrettG", replay_count=10, replay_date_after=datetime.datetime(2019, 6,1, tzinfo=datetime.timezone.utc))
]

# retrieve replay statistics
replay_stats = [
    ball_chaser.download_replay(replay["id"], 'replays')
    for replay in replays
]

# Create game frames using Carball

Note: you need boxcars-py 0.1.15 which is not available on pypi, so you need to compile it yourself

In [2]:
import carball
import glob
from carball.analysis.analysis_manager import AnalysisManager


replays = glob.glob("replays/*")
analysis_manager = carball.analyze_replay_file(replays[0])

Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 5268 by Darou


In [35]:
from rlgym.utils.gamestates import GameState, PhysicsObject, PlayerData
from rlgym.utils.math import euler_to_rotation, rotation_to_quaternion
from pandas import DataFrame, Series
from typing import List, Tuple
import numpy as np
from bisect import bisect_left

def get_orange_goal_frames(analysis_manager: AnalysisManager) -> List[int]:
    return [g.frame_number for g in analysis_manager.game.goals if g.player_team]
def get_blue_goal_frames(analysis_manager: AnalysisManager) -> List[int]:
    return [g.frame_number for g in analysis_manager.game.goals if not g.player_team]
def read_physics_object(row: Series, name: str) -> PhysicsObject:
    o = PhysicsObject()
    o.position = np.array([row['pos_x'][name], row['pos_y'][name], row['pos_z'][name]])
    o.linear_velocity = np.array([row['vel_x'][name], row['vel_y'][name], row['vel_z'][name]])
    o.angular_velocity = np.array([row['ang_vel_x'][name], row['ang_vel_y'][name], row['ang_vel_z'][name]])
    rotation_euler = np.array([row['rot_x'][name], row['rot_y'][name], row['rot_z'][name]])
    o.quaternion = euler_to_rotation(rotation_euler)

def quaternion_multiply(quaternion0, quaternion1):
    """ From https://stackoverflow.com/q/40915069
    
    Return multiplication of two quaternions.

    >>> q = quaternion_multiply([1, -2, 3, 4], [-5, 6, 7, 8])
    >>> numpy.allclose(q, [-44, -14, 48, 28])
    True

    """
    x0, y0, z0, w0 = quaternion0
    x1, y1, z1, w1 = quaternion1
    return np.array((
         x1*w0 + y1*z0 - z1*y0 + w1*x0,
        -x1*z0 + y1*w0 + z1*x0 + w1*y0,
         x1*y0 - y1*x0 + z1*w0 + w1*z0,
        -x1*x0 - y1*y0 - z1*z0 + w1*w0), dtype=np.float64)

def read_physics_object_inverted(row: Series, name: str, is_orange: bool):
    if not is_orange:
        return read_physics_object(row, name)
    
    o = PhysicsObject()
    o.pos = np.array([row['pos_x'][name], row['pos_y'][name], row['pos_z'][name]]) * np.array([-1, -1, 1])
    o.linear_velocity = np.array([row['vel_x'][name], row['vel_y'][name], row['vel_z'][name]]) * np.array([-1, -1, 1])
    o.angular_velocity = np.array([row['ang_vel_x'][name], row['ang_vel_y'][name], row['ang_vel_z'][name]]) * np.array([-1, -1, 1])
    rotation_euler = np.array([row['rot_x'][name], row['rot_y'][name], row['rot_z'][name]])
    o.quaternion = quaternion_multiply(rotation_to_quaternion(euler_to_rotation(rotation_euler)), np.array([0, 0, 0, -1]))



def get_game_states(analysis_manager: AnalysisManager):
    players : Tuple[str, bool] = [(p.name, p.is_orange) for p in analysis_manager.game.players]
    hit_frames : List[int] = [hit.frame_number for hit in  analysis_manager.protobuf_game.game_stats.hits]
    orange_goal_frames = get_orange_goal_frames(analysis_manager)
    blue_goal_frames = get_blue_goal_frames(analysis_manager)
    df = analysis_manager.get_data_frame()

    last_touch = None
    for row in df.iterrows():
        frame = int(row[0])
        row = row[1].unstack()

        orange_goals = sum(1 for g in orange_goal_frames if g <= frame)
        blue_goals = sum(1 for g in blue_goal_frames if g <= frame)

        player_objects = []
        player_index = {}
        for name, is_orange in players:
            player = PlayerData()
            player.car_data = read_physics_object(row, name)
            player.inverted_car_data = read_physics_object_inverted(row, name, is_orange)
            #TODO: player.on_ground
            player.boost_amount = row['boost'][name]

            player_index[name] = len(player_objects)
            player_objects.append(player)
            

        ball = read_physics_object(row, 'ball')
        touch_index = bisect_left(hit_frames, frame)
        last_touch = hit_frames[touch_index - 1] if touch_index > 0 else None
        
        for name, is_orange in players:
            state = GameState()
            state.game_type = 0
            state.blue_score = blue_goals
            state.orange_score = orange_goals
            state.ball = ball
            state.last_touch = last_touch

            # want to make sure player == players[0]
            state.players = [player_objects[player_index[name]]]
            for i in (i for i in range(len(player_objects)) if not i == player_index[name]):
                state.players.append(player_objects[i])
            

            yield state

for gs in get_game_states(analysis_manager):
    print(gs)
    break

********GAME STATE OBJECT********
Game Type: 0
Orange Score: 0
Blue Score: 0
PLAYERS: [<rlgym.utils.gamestates.player_data.PlayerData object at 0x000002CD839818C8>, <rlgym.utils.gamestates.player_data.PlayerData object at 0x000002CD8395E688>, <rlgym.utils.gamestates.player_data.PlayerData object at 0x000002CD8395EC08>, <rlgym.utils.gamestates.player_data.PlayerData object at 0x000002CD83961E48>]
BALL: None
INV_BALL: <rlgym.utils.gamestates.physics_object.PhysicsObject object at 0x000002CD8395ECC8>



In [10]:
x = analysis_manager.get_protobuf_data()

In [20]:
object_methods = [method_name for method_name in dir(x.ListFields()[0])
                  if callable(getattr(x.ListFields()[0], method_name))]
object_methods

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'count',
 'index']

In [31]:
hit_frames = [hit.frame_number for hit in  analysis_manager.protobuf_game.game_stats.hits]

In [32]:
hit_frames

[178,
 226,
 324,
 330,
 484,
 522,
 544,
 622,
 660,
 695,
 764,
 824,
 854,
 866,
 891,
 908,
 939,
 999,
 1125,
 1161,
 1213,
 1267,
 1304,
 1327,
 1397,
 1487,
 1552,
 1609,
 1918,
 1958,
 2065,
 2066,
 2158,
 2517,
 2549,
 2609,
 2637,
 2707,
 2995,
 2996,
 3122,
 3233,
 3298,
 3363,
 3409,
 3504,
 3507,
 3576,
 3704,
 3722,
 3725,
 3756,
 3800,
 3803,
 3804,
 3854,
 3915,
 3964,
 4264,
 4330,
 4628,
 4630,
 4702,
 4750,
 4800,
 4878,
 5019,
 5069,
 5125,
 5182,
 5212,
 5216,
 5268,
 5316,
 5339,
 5392,
 5472,
 5880,
 5882,
 5918,
 5999,
 6034,
 6094,
 6133,
 6164,
 6183,
 6253,
 6256,
 6295,
 6327,
 6370,
 6462,
 6486,
 6545,
 6548,
 6611,
 6686,
 6746,
 6757,
 6758,
 6831,
 6879,
 6949,
 7252,
 7309,
 7355,
 7407,
 7432,
 7472,
 7568,
 7586,
 7643,
 7670,
 7700,
 7714,
 7721,
 7745,
 7753,
 7763,
 7771,
 7951,
 7974,
 8012,
 8042,
 8053,
 8096,
 8125,
 8129,
 8130,
 8132,
 8143,
 8164,
 8174,
 8176,
 8233,
 8262,
 8273,
 8287,
 8293,
 8431,
 8478]

In [62]:
analysis_manager.events_creator.create_boostpad_events(analysis_manager.protobuf_game, analysis_manager.data_frame)aa